In [2]:
# run this cell to import nltk
import nltk
from os import getcwd
import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/anrui/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/anrui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)


In [4]:
filePath

'/Users/anrui/Desktop/projects/8 NLP learning/NLP_learning/../tmp2/'

In [5]:
nltk.data.path

['/Users/anrui/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data',
 '/Users/anrui/Desktop/projects/8 NLP learning/NLP_learning/../tmp2/']

In [141]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

In [7]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [8]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [9]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [10]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [12]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11337


In [13]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


<a name='1'></a>
## 1 - Logistic Regression 

<a name='1-1'></a>
### 1.1 - Sigmoid
You will learn to use logistic regression for text classification. 
* The sigmoid function is defined as: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability. 

<div style="width:image width px; font-size:100%; text-align:center;"><img src='./images/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> Figure 1 </div>

In [122]:
# UNQ_C1 GRADED FUNCTION: sigmoid
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE ###
    # calculate the sigmoid of z
    # import math
    h = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return h

In [142]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


In [143]:
# Test your function
w1_unittest.test_sigmoid(sigmoid)

 All tests passed


In [124]:
# # UNQ_C2 GRADED FUNCTION: gradientDescent
# def gradientDescent(x, y, theta, alpha, num_iters):
#     '''
#     Input:
#         x: matrix of features which is (m,n+1)
#         y: corresponding labels of the input matrix x, dimensions (m,1)
#         theta: weight vector of dimension (n+1,1)
#         alpha: learning rate
#         num_iters: number of iterations you want to train your model for
#     Output:
#         J: the final cost
#         theta: your final weight vector
#     Hint: you might want to print the cost to make sure that it is going down.
#     '''
#     ### START CODE HERE ###
#     # get 'm', the number of rows in matrix x
#     m = len(x)
    
#     for i in range(0, num_iters):
        
#         # get z, the dot product of x and theta
#         z = sum(np.dot(x, theta))
        
#         # get the sigmoid of z
#         h = sigmoid(z)
#         # print(f'h is {h}')

        

        
#         # calculate the cost function
#         # k = sum([(h-y[i])*x[i] for i in range(m)])
#         k=0
#         for i in range(m):
#             k += (h-float(y[i]))*x[i]
            
#         J = 1/(m*k)
#         print(type(k))
        

#         # update the weights theta
#         theta = theta - alpha * J
        
#     ### END CODE HERE ###
#     J = float(J)
#     return J, theta

In [164]:
# UNQ_C2 GRADED FUNCTION: gradientDescent
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    # get 'm', the number of rows in matrix x
    # m = len(x)
    m = x.shape[0]
    
    for i in range(num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x, theta)
        
        # get the sigmoid of z
        h = sigmoid(z)

        # calculate the cost function
        # J = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
        J = -1./m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T,np.log(1-h)))      
        
        # update the weights theta
        # theta = theta - alpha * (1/m) * sum(np.dot( x.T,(h - y)))
        theta = theta - (alpha/m) * np.dot(x.T,(h-y))
        
    return J, theta


In [180]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

TypeError: unsupported format string passed to numpy.ndarray.__format__

#### Expected output
```
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]
```

In [94]:
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# print(f"tmp_X is {tmp_X}")
# print(f"shape of tmp_X is {tmp_X.shape}")

tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

tmp_theta = np.random.rand(3, 1)
# print(f"tmp_theta is {tmp_theta}")
# print(f"shape of tmp_theta is {tmp_theta.shape}")

z = sum(np.dot(tmp_X, tmp_theta))
h = sigmoid(z)


# type(z)
# z
float(tmp_Y[0])





0.0

In [58]:
np.random.rand(3, 2)

array([[0.20329323, 0.25232574],
       [0.74382585, 0.19542948],
       [0.58135893, 0.97001999]])

In [19]:
x_1 = [[1,2,3], [2,3,1], [2,2,2], [3,7,9]]
len(x_1)

4

In [22]:
x_1.shape

AttributeError: 'list' object has no attribute 'shape'

In [23]:
my_matrix = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
len(my_matrix)

3

In [43]:
def cal_list_matrix(x, y):
    m, n = len(x), len(x[0])
    n, o = len(y), len(y[0])


    z = [[0] * o for _ in range(m)]
    # z =[]
    for i in range(m):
        for j in range(o):
            dot_product = 0
            for k in range(n):
                z[i][j] += x[i][k] * y[k][j]
             
    


    return z

In [46]:
x = [[1, 2], [4, 5]]
y = [[0,2], [1,3]]

z = cal_list_matrix(x,y)

print(z)

[[2, 8], [5, 23]]


In [49]:
x_2 = [1,2,3]
sum(x_2)

6

In [33]:
[x for x in range(2)]

[0, 1]

<a name='2'></a>
## 2 - Extracting the Features

* Given a list of tweets, extract the features and store them in a matrix. You will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet. 
* Then train your logistic regression classifier on these features.
* Test the classifier on a validation set. 

<a name='ex-3'></a>
### Exercise 3 - extract_features
Implement the extract_features function. 
* This function takes in a single tweet.
* Process the tweet using the imported `process_tweet` function and save the list of tweet words.
* Loop through each word in the list of processed words
    * For each word, check the 'freqs' dictionary for the count when that word has a positive '1' label. (Check for the key (word, 1.0)
    * Do the same for the count for when the word is associated with the negative label '0'. (Check for the key (word, 0.0).)

**Note:** In the implementation instructions provided above, the prediction of being positive or negative depends on feature vector which counts-in duplicate words - this is different from what you have seen in the lecture videos

In [181]:
# UNQ_C3 GRADED FUNCTION: extract_features
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements for [bias, positive, negative] counts
    x = np.zeros(3) 
    
    # bias term is set to 1
    x[0] = 1 
    
    ### START CODE HERE ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[1] += freqs.get((word,1.0),0)
        
        # increment the word count for the negative label 0
        x[2] += freqs.get((word,0.0),0)
        
    ### END CODE HERE ###
    
    x = x[None, :]  # adding batch dimension for further processing
    assert(x.shape == (1, 3))
    return x

In [182]:
# Check your function
# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [183]:
# Test your function
w1_unittest.test_extract_features(extract_features, freqs)

Wrong output values. Check how you are computing the positive or negative word count. 
	Expected: [[1.000e+00 3.133e+03 6.100e+01]].
	Got: [[1.00e+00 3.02e+03 6.10e+01]].
 7  Tests passed
 1  Tests failed


In [184]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [187]:
theta = [6e-08, 0.00053818, -0.0005583]

In [188]:
# UNQ_C4 GRADED FUNCTION: predict_tweet
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x, theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [189]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta))) 

I am happy -> 0.519140
I am bad -> 0.494347
this movie should have been great. -> 0.515845
great -> 0.515930
great great -> 0.531828
great great great -> 0.547661
great great great great -> 0.563399


In [190]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([0.82239168])

In [194]:
my_tweet = "it's nice to talk with ms Lu"
predict_tweet(my_tweet, freqs, theta)

array([0.50629562])

In [196]:
# UNQ_C5 GRADED FUNCTION: test_logistic_regression
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE ###
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy = sum(np.asarray(y_hat) == np.squeeze(test_y))/len(y_hat)

    ### END CODE HERE ###
    
    return accuracy

In [197]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9955


In [198]:
# Test your function
w1_unittest.unittest_test_logistic_regression(test_logistic_regression, freqs, theta)

 All tests passed


In [199]:
# Some error analysis done for you
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48848630	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418925	b"i'm play brain dot braindot"
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots http://t.co/aOKldo3GMj http://t.co/xWCM9qyRG5
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418925	b"i'm play brain dot braindot"
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots http://t.co/R2JBO8iNww http://t.co/ow5BBwdEMY
THE PROCESSED TWEET IS: ["i'm", 'play', 

In [200]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[0.48139357]
Negative sentiment


In [201]:
import numpy as np

def add(x, y):
    return x + y

add_vec = np.vectorize(add)

x = np.array([1, 2, 3, 4])
y = np.array([5, 6, 7, 8])

z = add_vec(x, y)
print(z)

[ 6  8 10 12]


In [202]:
x+y

array([ 6,  8, 10, 12])